In [1]:
# !pip install --upgrade huggingface_hub
!pip install huggingface_hub
#!pip install --upgrade fireworks-ai
#!pip install telebot
!pip install python-telegram-bot --upgrade
!pip install pyTelegramBotAPI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.5/673.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 20.2 MB/s eta 0:00:00


In [2]:
import telebot
import pandas as pd
import random
from huggingface_hub import InferenceClient
import logging
from collections import deque
from telebot import types
from PIL import Image
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

logging.basicConfig(level=logging.INFO)

In [4]:
!git clone https://github.com/frdaria/elfo_bot.git

Cloning into 'elfo_bot'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 31 (delta 4), reused 6 (delta 1), pack-reused 19 (from 1)
Receiving objects: 100% (31/31), 90.03 MiB | 12.68 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [5]:
df = pd.read_csv('elfo_bot/prompts_fandom.csv', on_bad_lines='skip')
chunks_films = pd.read_csv('elfo_bot/chunks_kinopoisk.csv', on_bad_lines='skip')
vectors_films = np.load('/content/elfo_bot/bg3_m3_kinopoisk_chunks.npy')
chunks_litres = pd.read_csv('elfo_bot/chunks_litres2.csv', on_bad_lines='skip')
vectors_litres = np.load('/content/elfo_bot/bg3_m3_litres_chunks2.npy')

chunks_films = chunks_films.rename(columns={"name":"title"})

In [6]:
# client_gemma = InferenceClient(
#     provider="hf-inference",
#     api_key=api_key,)
# client_picture = InferenceClient(
#     provider="fal-ai",
#     api_key=api_key,)
# client_mistral = InferenceClient(
#     provider="fireworks-ai",
#     api_key=api_key,
# )

In [6]:
#embedding model
model_rag = SentenceTransformer("BAAI/bge-m3")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [7]:
from huggingface_hub import InferenceClient
client = InferenceClient(
    provider="hf-inference",
    api_key=api_key,
)

prompt_system_dialogue = '''Ты - волшебный эльф по имени Элфо! Ты - добрый писатель и учитель, который хочет помочь ученикам лучше ориентироваться в классической литературе. Ты пишешь приличный, дружелюбный и захватывающий контент на русском языке. Иногда ты используешь эмодзи. Твои тексты просты для чтения, полны диалогов, юмора и шуток! Ты пишешь фанфики по мотивам русских классических книг. Ты используешь разных персонажей классических книг и даешь пользователю информацию про него. Когда пользователь дает тебе тему истории, ты начинаешь со слов "Однажды..." и заканчиваешь словами "Продолжим читать?". Если пользователь задает тебе вопрос, не связанный с фанфиками по классическим произведениям, ты отвечаешь: 'Я просто Элфо! Какую историю ты хочешь почитать?'''
#sys_prompt_gemma = """Ты волшебный Эльф, который пишет волшебные фанфики на русском языке. Твоя задача: 1) по заданному контексту написать историю. 2) Написать промпт для иллюстрации к этой истории. Используй шаблон: 'История:...Иллюстрация:...'Всегда следуй этому шаблону!!!"""
prompt_for_search_results = """Прочитай результаты поиска по запросу пользователя. Пользователь еще не видел этих результатов. Предложи мне использовать это произведение как основу фанфика."""
prompt_system_quest = """Ты преподаватель литературы и добрый ведущий квестов для своих учеников. Твоя задача протестировать ученика на глубокие знания русской классической литературы через геймификацию. Начни общение с учеников через введение в историю, а затем задавай ему наводящие вопросы, ответы на которые позволят продвинутся дальше по сюжету."""
prompt_system_fandom = """Ты добрый писатель по имени Элфо. Ты пишешь фанфики. Для этого ты помещаешь персонажей классической русской литературы в популярные фандомы. Прочитай запрос от пользователя и напиши кусочек истории. Твои истории приличные и дружелюбные. Запомни главное правило: любая твоя история должна содержать отсылку к русской литературе в образовательных целях."""
#prompt_system_recommendation = """Ты дружелюбный эльф-библиотекарь по имени Элфо, который помогает читателю выбрать книгу классической русской литературы. Если сообщение читателя содержит нецензурную лексику или элементы жестокости, ты отвечаешь 'Не слышал такого. Так какую книгу ты хочешь почитать?'. Твоя задача - рекомендовать книги по запросу пользователя, других вещей ты делать не умеешь."""
prompt_system_recommendation = """Прочитай сообщение от читателя, который написал тему книги которую он ищет, и прочитай результат поисковой выдачи на это сообщение. Представь, что пользователь еще не видел поисковую выдачу!!! Поэтому сформулируй логичный ответ пользователю опираясь исключительно на ответ поисковой выдачи. Если сообщение читателя содержит нецензурную лексику или элементы жестокости, ты отвечаешь 'Не слышал такого. Так какую книгу ты хочешь почитать?'. Твоя задача - рекомендовать книги по запросу пользователя, других вещей ты делать не умеешь."""
prompt_filmoknogi = """Ты дружелюбный эльф по имени Элфо, который умеет прекрасно находить аналогии между классическими русскими книгами и популярными фильмами. Ты применяешь свои знания, чтобы помочь своим ученикам запоминать героев книг и события, через создания ассоциаций с героями фильмов. Вот, какие тебе надо сравнить произведения на этот раз:  """
def gemma_generate(user_question: str, context: list, search_res, prompt_system_choice) -> str:

    if prompt_system_choice == 'Фанфик':
      mode_prompt = prompt_system_dialogue

    if prompt_system_choice == 'Квест':
      mode_prompt = prompt_system_quest

    if prompt_system_choice == 'Фандом':
      mode_prompt = prompt_system_fandom

    if prompt_system_choice == 'Рекомендация':
      mode_prompt = prompt_system_recommendation

    if prompt_system_choice == 'Фильмокниги':
      mode_prompt = prompt_filmoknogi

    # messages=[{
    #             "role": "system",
    #             "content": [{"type": "text", "text": mode_prompt}]
    #           }]
    messages=[
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": mode_prompt
                }
            ]
        }
    ]

    if context:
      messages.extend(context)
    print(f'DEBUG: {messages}')

    question = [{
      "role": "user",
      "content": [
        {
            "type": "text",
            "text": user_question
        }
      ]
      }]
    messages.extend(question)

    if search_res:
      extra_info_from_retriever = [{"role":"assistant",
             "content": f'{prompt_for_search_results}. Результат поиска: {search_res}'}]

      messages.extend(extra_info_from_retriever)
      print(f"[DEBUG] messages added")

    # completion = client.chat.completions.create(
    #             model="google/gemma-3-27b-it",
    #             #model="meta-llama/Meta-Llama-3-70B-Instruct",
    #             max_tokens=300,
    #             messages = messages)

    completion = client.chat.completions.create(
    model="google/gemma-3-27b-it",
    messages=messages,
    max_tokens=200,
    )

    gemma_answer = completion.choices[0].message.content.replace('\n\n', '').replace('##', '')
    return gemma_answer

a = gemma_generate("кто ты", [], '', 'Фанфик')
a

DEBUG: [{'role': 'system', 'content': [{'type': 'text', 'text': 'Ты - волшебный эльф по имени Элфо! Ты - добрый писатель и учитель, который хочет помочь ученикам лучше ориентироваться в классической литературе. Ты пишешь приличный, дружелюбный и захватывающий контент на русском языке. Иногда ты используешь эмодзи. Твои тексты просты для чтения, полны диалогов, юмора и шуток! Ты пишешь фанфики по мотивам русских классических книг. Ты используешь разных персонажей классических книг и даешь пользователю информацию про него. Когда пользователь дает тебе тему истории, ты начинаешь со слов "Однажды..." и заканчиваешь словами "Продолжим читать?". Если пользователь задает тебе вопрос, не связанный с фанфиками по классическим произведениям, ты отвечаешь: \'Я просто Элфо! Какую историю ты хочешь почитать?'}]}]


'Я просто Элфо! Какую историю ты хочешь почитать?\n'

In [12]:
# def dialogue_model(user_question: str, context: list) -> str:
#     messages = [
#                 {
#                   "role": "system",
#                   "content": prompt_system_dialogue
#                 },
#                ]
#     if context:
#          messages.extend(context)

#     question = [{'role':'user',
#                  'content': user_question}]
#     messages.extend(question)
#     completion = client_mistral.chat.completions.create(
#       model="mistralai/Mistral-Small-24B-Instruct-2501",
# 	    messages=messages,
#   	  max_tokens=200,
#     )
#     return completion.choices[0].message.content


def update_context(context, user_m, answer):
    new_data = [{'role': 'user', 'content': [{"type": "text", "text": user_m}]},
                {'role': 'system', 'content': [{"type": "text", "text": answer}]}]
    context.extend(new_data)
    return context

def get_fandom_prompt(df, fandom):
    if fandom in df.columns:
        prompts = df[fandom].dropna().tolist()
        if prompts:
            print(f"[DEBUG] Choosing fandom prompt")
            return random.choice(prompts)
        else:
            return "Для этого фандома нет промптов."
    else:
        return "Фандом не найден."


def process_dialogue_step(user_message, context, search_res, prompt_system_choice):
        try:
            answer = gemma_generate(user_message, list(context), search_res, prompt_system_choice)
        except Exception as e:
            logging.error(f"Ошибка в gemma_generate: {e}")
            return None, None
        if not answer:
            logging.error("Пустой ответ от gemma_generate")
            return None, None
        context = update_context(context, user_message, answer)
        logging.info(f"Updated context: {context}")
        return answer, context


# def gemma_generate(user_question: str, context: list, search_res, prompt_system_choice) -> str:

#     if prompt_system_choice == 'Фанфик':
#       mode_prompt = "ты няшка"

#     if prompt_system_choice == 'Квест':
#       mode_prompt = prompt_system_quest

#     if prompt_system_choice == 'Фандом':
#       mode_prompt = prompt_system_fandom

#     if prompt_system_choice == 'Рекомендация':
#       mode_prompt = prompt_system_recommendation

#     if prompt_system_choice == 'Фильмокниги':
#       mode_prompt = prompt_filmoknogi

#     # messages=[{
#     #             "role": "system",
#     #             "content": [{"type": "text", "text": mode_prompt}]
#     #           }]
#     messages=[
#         {
#             "role": "system",
#             "content": [
#                 {
#                     "type": "text",
#                     "text": mode_prompt
#                 }
#             ]
#         }
#     ]

#     print(f'DEBUG: {messages}')

#     question = [{
#       "role": "user",
#       "content": [
#         {
#             "type": "text",
#             "text": user_question
#         }
#       ]
#       }]
#     messages.extend(question)

#     if search_res:
#       extra_info_from_retriever = [{"role":"system",
#                                     "content": [{"type": "text",
#                                                  "text": f'{prompt_for_search_results}. Результат поиска: {search_res}'}
#                                                ]
#                                   }]

#       messages.extend(extra_info_from_retriever)
#       print(f"[DEBUG] messages added")

    # completion = client.chat.completions.create(
    #             model="google/gemma-3-27b-it",
    #             #model="meta-llama/Meta-Llama-3-70B-Instruct",
    #             max_tokens=300,
    #             messages = messages)

    # completion = client.chat.completions.create(
    # model="google/gemma-3-27b-it",
    # messages=messages,
    # max_tokens=500,
    # )

    # gemma_answer = completion.choices[0].message.content.replace('\n\n', '').replace('##', '')
    # return gemma_answer

def gemma_create_prompt(story) -> str:
    messages=[{
                "role": "user",
                "content": [{"type": "text", "text": f'Read the text and create a SIMPLE and SHORT prompt to illustrate the plot of the story. The prompt must be in English. Do not give any comments, your ouput must consist of the promplt only!!! Story: {story}'}]
              }]
    completion = client_gemma.chat.completions.create(
                model="google/gemma-3-27b-it",
                max_tokens=50,
                messages = messages)

    description = completion.choices[0].message.content.replace('\n\n', '').replace('##', '')
    return description

def picture_generate(picture_prompt):
    # output is a PIL.Image object
    image = client_picture.text_to_image(f'mystical style inspired by old fairy tales books, flat design. {picture_prompt}',
    model="stabilityai/stable-diffusion-3.5-large",)
    return image

def find_similar(query, data, vectors, top_k=3):
    query_vec = model_rag.encode(query)
    similarities = cosine_similarity([query_vec], vectors)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    results = []
    for idx in top_indices:
        title = data.iloc[idx]['title']
        description = data.iloc[idx]['description']
        results.append(f'Название: {title}, Описание: {description}')
    results_str = ''.join(results)
    return results_str

In [10]:
# results = find_similar("морские путешествия", chunks_films, vectors_films, model, top_k=3)
# print(results)

In [8]:
# Ограничиваем контекст последними n сообщениями
MAX_CONTEXT_LENGTH = 7
context_q = deque(maxlen=MAX_CONTEXT_LENGTH)
# dialog_check = 0
# answer = ''
user_states = {}


# @bot.message_handler(commands=["очистить"])
# def clear_context(message):
#     global context_q
#     context_q.clear()  # Очищаем контекст
#     bot.reply_to(message, "Контекст очищен. Начнём с чистого листа!")

fandoms = ["🌹💌 Евгений Онегин", "🧙✨ Властелин Колец", "👑⚔️ Игра престолов", "🔥🐲 Dragon Age", "🦹🏻❤️ Алые сердца Корё"]

def create_picture_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_fandom = types.KeyboardButton("Изобразить")
    button_main_keyboard = types.KeyboardButton("В начало")
    keyboard.add(button_fandom, button_main_keyboard)
    return keyboard

# Создаем клавиатуру
def create_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_fandom = types.KeyboardButton("📚 Выбрать фандом")
    button_interactive_fanfic = types.KeyboardButton("Интерактивный фанфик ☕️")
    button_quest = types.KeyboardButton("Квест 🐾")
    button_matrix = types.KeyboardButton("Матрица фильмов и книг 🔮")
    keyboard.add(button_fandom, button_interactive_fanfic, button_quest, button_matrix)
    return keyboard

def create_matrix_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_recommendation = types.KeyboardButton("Что бы почитать?")
    button_create_assosiations = types.KeyboardButton("Книга vs Фильм")
    button_main_keyboard = types.KeyboardButton("В начало")
    keyboard.add(button_recommendation, button_create_assosiations, button_main_keyboard)
    return keyboard

@bot.message_handler(func=lambda message: message.text == "Матрица фильмов и книг 🔮")
def matrix_info(message):
    global dialog_check, context_q
    context_q.clear()
    bot.reply_to(message, """Выбери режим:
                          1) Режим 'Что бы почитать?' - дает рекомендации для чтения классической литературы.
                          2) Режим 'Книга vs Фильм' - найдет аналог книге по смыслу среди фильмов.
                          """, reply_markup=create_matrix_keyboard())

@bot.message_handler(func=lambda message: message.text == "Что бы почитать?")
def book_recommendation(message):
    global dialog_check, context_q
    context_q.clear()
    bot.reply_to(message, """Напиши пару слов о том, про что должна быть книга :)""", reply_markup=create_matrix_keyboard())
    dialog_check = 5
    print(f"[DEBUG] Setting dialog_check to 5")

@bot.message_handler(func=lambda message: message.text == "Книга vs Фильм")
def book_vs_film(message):
    global dialog_check, context_q
    context_q.clear()
    bot.reply_to(message, """В этом режиме ты можешь найти книгу русской литературы, сюжет которой напоминает твой любимый фильм. Ты можешь использовать этот режим для создания ассоциаций с фильмами и лучше запоминать произведения! Напиши название любой книги.
                          """, reply_markup=create_matrix_keyboard())
    dialog_check = 6
    print(f"[DEBUG] Setting dialog_check to 6")


def create_options_for_interactive_acts():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_sem_search = types.KeyboardButton("Включить поиск")  # ищем информацию, если модель не справляется
    button_main_keyboard = types.KeyboardButton("В начало")
    keyboard.add(button_sem_search, button_main_keyboard)
    return keyboard

def create_fandom_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_eo = types.KeyboardButton("🌹💌 Евгений Онегин")
    button_lotr = types.KeyboardButton("🧙✨ Властелин Колец")
    button_ip = types.KeyboardButton("👑⚔️ Игра престолов")
    button_da = types.KeyboardButton("🔥🐲 Dragon Age")
    button_core = types.KeyboardButton("🦹🏻❤️ Алые сердца Корё")
    button_main_keyboard = types.KeyboardButton("В начало")
    keyboard.add(button_eo, button_lotr, button_ip, button_da, button_core, button_main_keyboard)
    return keyboard


@bot.message_handler(func=lambda message: message.text == "Изобразить")
def handle_picture_request(message):
    global context_q
    if context_q:
        last_answer = context_q[-1]['content']
        if context_q[-1]['role'] == 'system':
            pic_prompt = gemma_create_prompt(last_answer)
        else:
            pic_prompt = context_q[-2]['content'] if len(context_q) > 1 else ""
    else:
        pic_prompt = ""
    try:
        image = picture_generate(pic_prompt)
        image.save('generated_image.jpg')
        bot.send_photo(message.chat.id, open('generated_image.jpg', 'rb'))
        bot.send_message(message.chat.id, "Выберите фандом:", reply_markup=create_keyboard())
    except Exception as e:
        print(f"Ошибка генерации: {e}")
        bot.reply_to(message, "Не удалось создать изображение 😔")


@bot.message_handler(func=lambda message: message.text == "📚 Выбрать фандом")
def choose_fandom(message):
    global dialog_check
    global fandom_prompt
    bot.reply_to(message, "Выбери фандом:", reply_markup=create_fandom_keyboard())

@bot.message_handler(func=lambda message: message.text == "Квест 🐾")
def handle_quest(message):
    global dialog_check, context_q
    context_q.clear()
    bot.reply_to(message, "Какое произведение нужно вызубрить?")
    dialog_check = 3
    print(f"[DEBUG] Setting dialog_check to 3")

@bot.message_handler(func=lambda message: message.text == "Интерактивный фанфик ☕️")
def handle_interactive_fanfic(message):
    global dialog_check, context_q
    context_q.clear()
    bot.reply_to(message, "Какую книгу возьмем за основу фанфика?", reply_markup=create_options_for_interactive_acts())
    dialog_check = 1
    print(f"[DEBUG] Setting dialog_check to 1")

@bot.message_handler(func=lambda message: message.text == "Включить поиск")
def handle_interactive_fanfic(message):
    global dialog_check
    bot.reply_to(message, "Опиши сюжет книги, которую нужно найти :)")
    dialog_check = 2
    print(f"[DEBUG] Setting dialog_check to 2")

@bot.message_handler(func=lambda message: message.text == "В начало")
def choose_fandom(message):
    global dialog_check
    context_q.clear()
    answer = ""
    search_result = ""
    search_result_kino = ""
    search_result_recommendation = ""
    fandom_prompt = ""
    print(f"[DEBUG] Data cleaned")
    bot.reply_to(message, "Хорошо! Напиши 'да', если захочешь еще почитать!", reply_markup=create_keyboard())
    dialog_check = 0
    print(f"[DEBUG] dialog_check set to {dialog_check}")

@bot.message_handler(func=lambda message: message.text == "📚 Выбрать фандом" or message.text in fandoms)
def handle_fandom_flow(message):
    global dialog_check, fandom_prompt, df, context_q
    context_q.clear()

    # Если пользователь начал выбор фандома
    if message.text == "📚 Выбрать фандом":
        bot.reply_to(message, "Выбери фандом:", reply_markup=create_fandom_keyboard())
        return

    # Если пользователь выбрал конкретный фандом
    if message.text in fandoms:
        fandom = message.text
        print(f"[DEBUG] Выбран фандом: {fandom}")

        # Генерируем промпт для фандома
        fandom_prompt = get_fandom_prompt(df, fandom)
        print(f"[DEBUG] Промпт фандома: {fandom_prompt}")

        # Обрабатываем немедленно
        bot.reply_to(message, "⏳ Генерирую историю...", reply_markup=types.ReplyKeyboardRemove())
        answer, context_q = process_dialogue_step(fandom_prompt, [], '', 'Фандом')

        if answer:
            bot.reply_to(message, f"{answer}\nПродолжать?", reply_markup=create_picture_keyboard())
            dialog_check = 1  # Переходим в режим продолжения
        else:
            bot.reply_to(message, "Ошибка генерации 😞")
            dialog_check = 0

        print(f"[DEBUG] Состояние диалога: {dialog_check}")

# @bot.message_handler(func=lambda message: message.text == "📚 Выбрать фандом")
# def choose_fandom(message):
#     global dialog_check, fandom_prompt
#     bot.reply_to(message, "Выбери фандом:", reply_markup=create_fandom_keyboard())

# @bot.message_handler(func=lambda message: message.text in fandoms)
# def handle_fandom(message):
#     global dialog_check, fandom_prompt, df
#     fandom = message.text
#     print(f"[DEBUG] Fandom selected: {fandom}")
#     fandom_prompt = get_fandom_prompt(df, fandom)
#     print(f"[DEBUG] Fandom prompt created")
#     bot.reply_to(message, "⏳ Обрабатываю выбор...", reply_markup=create_fandom_keyboard())
#     dialog_check = 4
#     print(f"[DEBUG] dialog_check={dialog_check}")

def process_state_4(message):
    global dialog_check, context_q
    if dialog_check == 4:
        print(f"[DEBUG] Обработка состояния 4")
        answer, context_q = process_dialogue_step(fandom_prompt, context_q, '', 'Фандом')
        if answer:
            bot.reply_to(message.chat.id, f"{answer}\nПродолжать?", reply_markup=create_picture_keyboard())
            dialog_check = 1
        else:
            bot.reply_to(message.chat.id, "Ошибка генерации")
            dialog_check = 0

    # if dialog_check == 4:
    #   answer, context_q = process_dialogue_step(fandom_prompt, context_q, '', 'Фандом')
    # if answer is not None:
    #     bot.reply_to(message, f"{answer}\nПродолжать?", reply_markup=create_picture_keyboard())

In [9]:
@bot.message_handler(func=lambda message: True)
def answer_the_question(message):
    global context_q, search_result_kino, search_result_recommendation, search_result
    global search_result
    global dialog_check
    global fandom_prompt
    global answer
    global df, user_wish

    print(f"[DEBUG] dialog_check={dialog_check}, context_q={len(context_q)}")
    user_message = message.text

    # Проверка на команду завершения
    if user_message.lower() in ["стоп", "хватит", "закончим", "очистить", "нет"]:
        bot.reply_to(message, "Хорошо, давай закончим. Если захочешь ещё историй, просто напиши!")
        context_q.clear()  # Очищаем контекст
        dialog_check = 0
        return

    if dialog_check == 0:
        bot.reply_to(message, "Привет! Я Элфо! Что почитаем сегодня?", reply_markup=create_keyboard())
        #dialog_check = 1
        return

    if dialog_check == 2:
        query = user_message
        print(f"[DEBUG] query={query}")
        search_result = find_similar(query, chunks_litres, vectors_litres, top_k=1)
        print(f"[DEBUG] search_result={search_result}")
        dialog_check = 1
        print(f"[DEBUG] dialog_check={dialog_check}")

    if dialog_check == 1:
        book_prompt = f'За основу фанфика возьмем: {user_message}' if not context_q else user_message
        print(f"[DEBUG] book_prompt={book_prompt}")
        additional_prompt = ""
        if search_result:
          additional_prompt = search_result
        print(f"[DEBUG] additional_prompt={additional_prompt}")
        answer, context_q = process_dialogue_step(book_prompt, context_q, additional_prompt, prompt_system_choice='Фанфик')

        if answer is not None:
            bot.reply_to(message, answer, reply_markup=create_keyboard())
            bot.reply_to(message, "Продолжаем?")
            search_result = ""

    if dialog_check == 3:
        quest_book = f'Хочу пройти квест по этому произведению: {user_message}' if not context_q else user_message
        print(f"[DEBUG] quest_book={quest_book}")
        answer, context_q = process_dialogue_step(quest_book, context_q, '', 'Квест')

        if answer is not None:
            bot.reply_to(message, answer, reply_markup=create_keyboard())
            bot.reply_to(message, "Продолжаем?")

    if dialog_check == 4:
        print(f"[DEBUG] moved to part 4")
        answer, context_q = process_dialogue_step(fandom_prompt, context_q, '', 'Фандом')
        if answer:
          print(f"[DEBUG] Generated answer: {answer[:50]}...")
          bot.reply_to(message, f"{answer}\nПродолжать?", reply_markup=create_picture_keyboard())
          dialog_check = 1

    if dialog_check == 5:
        print(f"[DEBUG] moved to part 5")
        user_wish = f'Я бы хотел почитать книгу про: {user_message}' if not context_q else user_message
        print(f"[DEBUG] user_message = {user_wish}")
        if not context_q:
          search_result_recommendation = find_similar(user_message, chunks_litres, vectors_litres, top_k=1)
          print(f"[DEBUG] search_result_recommendation = {search_result_recommendation}")
        else:
          search_result_recommendation = ''
        answer, context_q = process_dialogue_step(user_wish, context_q, search_result_recommendation, 'Рекомендация')
        if answer:
          print(f"[DEBUG] Generated answer: {answer[:50]}...")
          bot.reply_to(message, f"{answer}", reply_markup=create_matrix_keyboard())

    if dialog_check == 6:
        print(f"[DEBUG] moved to part 6")
        users_fav_film = f'Книга: {user_message}' if not context_q else user_message
        print(f"[DEBUG] users_fav_film = {users_fav_film}")
        if not context_q:
          search_result_kino = find_similar(user_message, chunks_films, vectors_films, top_k=1)
          search_result_kino = f'Фильм, с которым надо сравнить: {search_result_kino}'
        else:
          search_result_kino = ''
        print(f"[DEBUG] search_result_kino = {search_result_kino}")
        answer, context_q = process_dialogue_step(users_fav_film, context_q, search_result_kino, 'Фильмокниги')
        if answer:
          print(f"[DEBUG] Generated answer: {answer[:50]}...")
          bot.reply_to(message, f"{answer}", reply_markup=create_matrix_keyboard())

    # if dialog_check == 7:
    #     context_q.clear()
    #     answer = ""
    #     search_result = ""
    #     search_result_kino = ""
    #     search_result_recommendation = ""
    #     fandom_prompt = ""
    #     print(f"[DEBUG] Data cleaned")
    #     dialog_check = 0

    return

In [10]:
dialog_check = 0
context_q = []
answer = ""
search_result = ""
search_result_kino = ""
search_result_recommendation = ""
fandom_prompt = ""

In [13]:
bot.polling(none_stop=True)

[DEBUG] dialog_check=6, context_q=0
[DEBUG] moved to part 6
[DEBUG] users_fav_film = Книга: пиковая дама
[DEBUG] search_result_kino = Фильм, с которым надо сравнить: Название: Чего хотят женщины, Описание: сексапильный ник маршал находится на пике карьеры в рекламном агентстве и рассчитывает на повышение, но его место занимает дарси макгуайр. однажды с ником происходит несчастный случай, после которого он начинает читать женские мысли. с помощью этого дара плейбой хочет вернуть утраченные позиции на работе, но вместо этого находит свою любовь.
DEBUG: [{'role': 'system', 'content': [{'type': 'text', 'text': 'Ты дружелюбный эльф по имени Элфо, который умеет прекрасно находить аналогии между классическими русскими книгами и популярными фильмами. Ты применяешь свои знания, чтобы помочь своим ученикам запоминать героев книг и события, через создания ассоциаций с героями фильмов. Вот, какие тебе надо сравнить произведения на этот раз:  '}]}]
[DEBUG] messages added
[DEBUG] Generated answer: О

In [ ]:
def send_message_with_picture_keyboard(chat_id, text):
    bot.send_message(chat_id=chat_id, text=text, reply_markup=create_picture_keyboard())

In [ ]:
from telebot import types
from collections import deque
import logging
import random


# Инициализация контекста
MAX_CONTEXT_LENGTH = 5
user_contexts = {}
dialog_check = 0

# Создаем клавиатуру
def create_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_random = types.KeyboardButton("🎲 Рандомный сюжет от Элфо")
    button_fandom = types.KeyboardButton("📚 Выбрать фандом")
    button_continue = types.KeyboardButton("➡️ Продолжить историю")
    keyboard.add(button_random, button_fandom, button_continue)
    return keyboard

# Клавиатура для выбора фандома
def create_fandom_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_hp = types.KeyboardButton("🪄 Гарри Поттер")
    button_lotr = types.KeyboardButton("🧙 Властелин Колец")
    button_sw = types.KeyboardButton("⭐ Звёздные Войны")
    keyboard.add(button_hp, button_lotr, button_sw)
    return keyboard

# Функции для промптов
def get_random_prompt():
    prompts = [
        "Жил-был дракон, который любил чай...",
        "Однажды в далёкой галактике...",
        "В маленькой деревне жил волшебник, который боялся магии..."
    ]
    return random.choice(prompts)

def get_fandom_prompt(fandom):
    prompts = {
        "🪄 Гарри Поттер": "Напиши историю о Гарри Поттере, который обнаружил новое заклинание...",
        "🧙 Властелин Колец": "Напиши историю о Фродо, который нашёл новое кольцо...",
        "⭐ Звёздные Войны": "Напиши историю о Люке Скайуокере, который встретил нового дроида..."
    }
    return prompts.get(fandom, "Фандом не найден.")

# Управление контекстом
def get_user_context(user_id):
    if user_id not in user_contexts:
        user_contexts[user_id] = deque(maxlen=MAX_CONTEXT_LENGTH)
    return user_contexts[user_id]

# Обработка нажатий на кнопки
@bot.message_handler(func=lambda message: message.text == "🎲 Рандомный сюжет от Элфо")
def random_story(message):
    prompt = get_random_prompt()
    bot.reply_to(message, prompt)

@bot.message_handler(func=lambda message: message.text == "📚 Выбрать фандом")
def choose_fandom(message):
    bot.reply_to(message, "Выбери фандом:", reply_markup=create_fandom_keyboard())

@bot.message_handler(func=lambda message: message.text in ["🪄 Гарри Поттер", "🧙 Властелин Колец", "⭐ Звёздные Войны"])
def handle_fandom(message):
    user_id = message.from_user.id
    fandom = message.text

    # Получаем промпт для выбранного фандома
    prompt = get_fandom_prompt(fandom)

    # Отправляем запрос к модели
    try:
        answer = dialogue_model(prompt, list(get_user_context(user_id)))
    except Exception as e:
        logging.error(f"Ошибка в dialogue_model: {e}")
        bot.reply_to(message, "Произошла ошибка. Попробуйте ещё раз.")
        return

    if not answer:
        logging.error("Пустой ответ от dialogue_model")
        bot.reply_to(message, "Кажется, Элфо заснул... Попробуйте ещё раз.")
        return

    # Отправляем ответ пользователю
    bot.reply_to(message, answer)

    # Обновляем контекст
    context_q = get_user_context(user_id)
    context_q.append({"user": prompt, "assistant": answer})

    # Возвращаем основную клавиатуру
    bot.reply_to(message, "Что будет дальше?", reply_markup=create_keyboard())

@bot.message_handler(func=lambda message: message.text == "➡️ Продолжить историю")
def continue_story(message):
    user_id = message.from_user.id
    context_q = get_user_context(user_id)
    if context_q:
        bot.reply_to(message, "Продолжи историю:")
    else:
        bot.reply_to(message, "Сначала выбери сюжет или фандом!", reply_markup=create_keyboard())

# Основная логика
@bot.message_handler(func=lambda message: True)
def answer_the_question(message):
    global dialog_check

    user_id = message.from_user.id
    context_q = get_user_context(user_id)

    def process_dialogue_step(user_message, context):
        try:
            answer = dialogue_model(user_message, list(context))
        except Exception as e:
            logging.error(f"Ошибка в dialogue_model: {e}")
            bot.reply_to(message, "Произошла ошибка. Попробуйте ещё раз.")
            return None, None

        if not answer:
            logging.error("Пустой ответ от dialogue_model")
            bot.reply_to(message, "Кажется, Элфо заснул... Попробуйте ещё раз.")
            return None, None

        bot.reply_to(message, answer)
        context.append({"user": user_message, "assistant": answer})
        logging.info(f"Updated context: {context}")

        return answer, context

    user_message = message.text

    if user_message.lower() in ["очистить", "стоп", "хватит"]:
        context_q.clear()
        dialog_check = 0
        bot.reply_to(message, "Контекст очищен. Начнём с чистого листа!", reply_markup=create_keyboard())
        return

    if dialog_check == 0:
        bot.reply_to(message, "Привет! Я Элфо! Какую историю ты хочешь почитать?", reply_markup=create_keyboard())
        dialog_check = 1
        return

    if dialog_check == 1:
        answer, context_q = process_dialogue_step(user_message, context_q)

        if answer is not None:
            bot.reply_to(message, answer)
            bot.reply_to(message, "Что будет дальше?", reply_markup=create_keyboard())
        return

# Запуск бота
bot.polling()

In [ ]:
bot.polling(none_stop=True)

In [ ]:



# # @bot.message_handler(commands=['start'])
# # def greet_user(message):
# #   bot.send_message(message.chat.id, "")

# import logging
# logging.basicConfig(level=logging.INFO)

# dialog_check = 0
# context_q = []
# question = ""
# @bot.message_handler(func=lambda message: True)
# def answer_the_question(message):

#   global dialog_check
#   global context_q
#   global question

#   if dialog_check == 0:
#     bot.reply_to(message, "Привет! Я Элфо! Какую историю, ты хочешь почитать?")
#     dialog_check = 1
#     return

#   if dialog_check == 1:
#     user_message = message.text
#     try:
#       answer = dialogue_model(user_message, context_q)
#     except Exception as e:
#       logging.error(f"Ошибка в dialogue_model: {e}")
#       bot.reply_to(message, "Произошла ошибка. Попробуйте ещё раз.")
#       return


#     bot.reply_to(message, answer)

#     if not answer:
#       logging.error("Пустой ответ от dialogue_model")
#       bot.reply_to(message, "Кажется, Элфо заснул... Попробуйте ещё раз.")
#       return

#     context_q = update_context(context_q, user_message, answer)

#     logging.info(f"Updated context: {context_q}")
#     dialog_check = 2
#     return

#   if dialog_check == 2:
#     user_message = message.text
#     answer = dialogue_model(user_message, context_q)
#     bot.reply_to(message, answer)
#     ## обновляем контекст
#     context_q = update_context(context_q, user_message, answer)
#     print(context_q)
#     dialog_check = 0
#     return

In [ ]:
## вот тут все хорошо работало:

# @bot.message_handler(func=lambda message: True)
# def answer_the_question(message):
#     global context_q
#     global dialog_check

#     def process_dialogue_step(user_message, context):
#         try:
#             answer = dialogue_model(user_message, list(context))  # Преобразуем deque в список
#         except Exception as e:
#             logging.error(f"Ошибка в dialogue_model: {e}")
#             bot.reply_to(message, "Произошла ошибка. Попробуйте ещё раз.")
#             return None, None

#         if not answer:
#             logging.error("Пустой ответ от dialogue_model")
#             bot.reply_to(message, "Кажется, Элфо заснул... Попробуйте ещё раз.")
#             return None, None

#         bot.reply_to(message, answer)
#         context = update_context(context, user_message, answer)
#         logging.info(f"Updated context: {context}")

#         return answer, context

#     user_message = message.text

#     # Проверка на команду завершения
#     if user_message.lower() in ["стоп", "хватит", "закончим"]:
#         bot.reply_to(message, "Хорошо, давай закончим. Если захочешь ещё историй, просто напиши!")
#         context_q.clear()  # Очищаем контекст
#         return

#     if dialog_check == 0:
#         bot.reply_to(message, "Привет! Я Элфо! Какую историю ты хочешь почитать?")
#         dialog_check = 1
#         return

#     # Обработка шага диалога
#     if dialog_check == 1:
#       answer, context_q = process_dialogue_step(user_message, context_q)


#     if context_q:  # Если ответ успешно сгенерирован
#         bot.reply_to(message, "Что будет дальше?😜")  # Подсказка для продолжения

#     return